import what we need

In [14]:
from pathlib import Path
from pprint import pprint

import tensorflow as tf
import torch
# import tweetynet

cutting and pasting model for now to not deal with fixing my environment

In [15]:
import torch
from torch import nn
from torch.nn import functional as F


class Conv2dTF(nn.Conv2d):
    """Conv2d with padding behavior from Tensorflow

    adapted from
    https://github.com/mlperf/inference/blob/16a5661eea8f0545e04c86029362e22113c2ec09/others/edge/object_detection/ssd_mobilenet/pytorch/utils.py#L40
    as referenced in this issue:
    https://github.com/pytorch/pytorch/issues/3867#issuecomment-507025011

    used to maintain behavior of original implementation of TweetyNet that used Tensorflow 1.0 low-level API
    """
    def __init__(self, *args, **kwargs):
        super(Conv2dTF, self).__init__(*args, **kwargs)
        self.padding = kwargs.get("padding", "SAME")

    def _compute_padding(self, input, dim):
        input_size = input.size(dim + 2)
        filter_size = self.weight.size(dim + 2)
        effective_filter_size = (filter_size - 1) * self.dilation[dim] + 1
        out_size = (input_size + self.stride[dim] - 1) // self.stride[dim]
        total_padding = max(
            0, (out_size - 1) * self.stride[dim] + effective_filter_size - input_size
        )
        additional_padding = int(total_padding % 2 != 0)

        return additional_padding, total_padding

    def forward(self, input):
        if self.padding == "VALID":
            return F.conv2d(
                input,
                self.weight,
                self.bias,
                self.stride,
                padding=0,
                dilation=self.dilation,
                groups=self.groups,
            )
        rows_odd, padding_rows = self._compute_padding(input, dim=0)
        cols_odd, padding_cols = self._compute_padding(input, dim=1)
        if rows_odd or cols_odd:
            input = F.pad(input, [0, cols_odd, 0, rows_odd])

        return F.conv2d(
            input,
            self.weight,
            self.bias,
            self.stride,
            padding=(padding_rows // 2, padding_cols // 2),
            dilation=self.dilation,
            groups=self.groups,
        )


class TweetyNet(nn.Module):
    def __init__(self,
                 num_classes,
                 input_shape=(1, 513, 88),
                 conv1_filters=32,
                 conv1_kernel_size=(5, 5),
                 conv2_filters=64,
                 conv2_kernel_size=(5, 5),
                 pool1_size=(8, 1),
                 pool1_stride=(8, 1),
                 pool2_size=(8, 1),
                 pool2_stride=(8, 1),
                 ):
        """initialize TweetyNet model

        Parameters
        ----------
        num_classes : int
            number of classes to predict, e.g., number of syllable classes in an individual bird's song
        input_shape : tuple
            with 3 elements corresponding to dimensions of spectrogram windows: (channels, frequency bins, time bins).
            i.e. we assume input is a spectrogram and treat it like an image, typically with one channel,
            the rows are frequency bins, and the columns are time bins. Default is (1, 513, 88).
        conv1_filters : int
            Number of filters in first convolutional layer. Default is 32.
        conv1_kernel_size : tuple
            Size of kernels, i.e. filters, in first convolutional layer. Default is (5, 5).
        conv2_filters : int
            Number of filters in second convolutional layer. Default is 64.
        conv2_kernel_size : tuple
            Size of kernels, i.e. filters, in second convolutional layer. Default is (5, 5).
        pool1_size : two element tuple of ints
            Size of sliding window for first max pooling layer. Default is (1, 8)
        pool1_stride : two element tuple of ints
            Step size for sliding window of first max pooling layer. Default is (1, 8)
        pool2_size : two element tuple of ints
            Size of sliding window for second max pooling layer. Default is (1, 8),
        pool2_stride : two element tuple of ints
            Step size for sliding window of second max pooling layer. Default is (1, 8)
        """
        super().__init__()
        self.num_classes = num_classes
        self.input_shape = input_shape

        self.cnn = nn.Sequential(
            Conv2dTF(in_channels=self.input_shape[0],
                     out_channels=conv1_filters,
                     kernel_size=conv1_kernel_size,
                     padding='same'
                     ),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=pool1_size,
                         stride=pool1_stride),
            Conv2dTF(in_channels=conv1_filters,
                      out_channels=conv2_filters,
                      kernel_size=conv2_kernel_size,
                     padding = 'same'
                     ),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=pool2_size,
                         stride=pool2_stride),
        )

        # determine number of features in output after stacking channels
        # we use the same number of features for hidden states
        # note self.num_hidden is also used to reshape output of cnn in self.forward method
        batch_shape = tuple((1,) + input_shape)
        tmp_tensor = torch.rand(batch_shape)
        tmp_out = self.cnn(tmp_tensor)
        channels_out, freqbins_out = tmp_out.shape[1], tmp_out.shape[2]
        self.num_rnn_features = channels_out * freqbins_out

        self.rnn = nn.LSTM(input_size=self.num_rnn_features,
                           hidden_size=self.num_rnn_features,
                           num_layers=1,
                           dropout=0,
                           bidirectional=True)

        # for self.fc, in_features = num_rnn_features * 2
        # because LSTM is bidirectional
        # so we get features forward + features backward as output
        self.fc = nn.Linear(self.num_rnn_features * 2, num_classes)

    def forward(self, x):
        features = self.cnn(x)
        # stack channels so that dimension order is (batch, num_rnn_features, num time bins)
        features = features.view(features.shape[0], self.num_rnn_features, -1)
        # now switch dimensions for feeding to rnn,
        # so dimension order is (num time bins, batch size, num_rnn_features)
        features = features.permute(2, 0, 1)
        rnn_output, (hidden, cell_state) = self.rnn(features)
        # permute back to (batch, time bins, features)
        # so we can project features down onto number of classes
        rnn_output = rnn_output.permute(1, 0, 2)
        logits = self.fc(rnn_output)
        # permute yet again
        # so that dimension order is (batch, classes, time steps)
        # because this is order that loss function expects
        return logits.permute(0, 2, 1)


instantiate model so we can look at size of parameter arrays.

Pick 8 classes because I can see from loaded checkpoint (below) that was the number of classes at output.

In [16]:
a_net = TweetyNet(num_classes=8)

load a checkpoint.

Following this blog post from Hugging Face.
https://medium.com/huggingface/from-tensorflow-to-pytorch-265f40ef2a28

Seems to work for `tensorflow` 1.13

In [23]:
ckpt_path = Path(
    '~/Documents/data/birdsong/BFSongRepository/vak/bl26lb16/'
    'results_191111_084054/TweetyNet/checkpoint_TweetyNet'
).expanduser()

In [24]:
tf_vars = tf.train.list_variables(str(ckpt_path))
pprint(tf_vars)

[('inference/Projection/Variable', [128, 8]),
 ('inference/Projection/Variable_1', [128, 8]),
 ('inference/Projection/Variable_2', [8]),
 ('inference/bidirectional_rnn/bw/basic_lstm_cell/bias', [512]),
 ('inference/bidirectional_rnn/bw/basic_lstm_cell/kernel', [256, 512]),
 ('inference/bidirectional_rnn/fw/basic_lstm_cell/bias', [512]),
 ('inference/bidirectional_rnn/fw/basic_lstm_cell/kernel', [256, 512]),
 ('inference/conv1/bias', [32]),
 ('inference/conv1/kernel', [5, 5, 1, 32]),
 ('inference/conv2/bias', [64]),
 ('inference/conv2/kernel', [5, 5, 32, 64]),
 ('optimize/beta1_power', []),
 ('optimize/beta2_power', []),
 ('optimize/global_step', []),
 ('optimize/inference/Projection/Variable/Adam', [128, 8]),
 ('optimize/inference/Projection/Variable/Adam_1', [128, 8]),
 ('optimize/inference/Projection/Variable_1/Adam', [128, 8]),
 ('optimize/inference/Projection/Variable_1/Adam_1', [128, 8]),
 ('optimize/inference/Projection/Variable_2/Adam', [8]),
 ('optimize/inference/Projection/Var

only keep the inference vars,
we're not worried about the state of the optimizer

In [10]:
inf_vars = [
    tf_var for tf_var in tf_vars if tf_var[0].startswith('inference')
]

In [11]:
inf_vars

[('inference/Projection/Variable', [128, 8]),
 ('inference/Projection/Variable_1', [128, 8]),
 ('inference/Projection/Variable_2', [8]),
 ('inference/bidirectional_rnn/bw/basic_lstm_cell/bias', [512]),
 ('inference/bidirectional_rnn/bw/basic_lstm_cell/kernel', [256, 512]),
 ('inference/bidirectional_rnn/fw/basic_lstm_cell/bias', [512]),
 ('inference/bidirectional_rnn/fw/basic_lstm_cell/kernel', [256, 512]),
 ('inference/conv1/bias', [32]),
 ('inference/conv1/kernel', [5, 5, 1, 32]),
 ('inference/conv2/bias', [64]),
 ('inference/conv2/kernel', [5, 5, 32, 64])]

Actually not so daunting now that we see there's just a few parameters.

CNN will be easiest to deal with.

In [37]:
cnn_params = list(a_net.cnn.named_parameters())

In [45]:
for p in cnn_params:
    print(p[0], p[1].shape)

0.weight torch.Size([32, 1, 5, 5])
0.bias torch.Size([32])
3.weight torch.Size([64, 32, 5, 5])
3.bias torch.Size([64])


In [46]:
tf_var_cnn_param_map = {
    'inference/conv1/kernel': a_net.cnn[0].weight,
    'inference/conv1/bias': a_net.cnn[0].bias,
    'inference/conv2/kernel': a_net.cnn[3].weight,
    'inference/conv2/bias': a_net.cnn[3].bias,    
}

Will need to change dimension order

**Also need to rotate / transpose arrays because input to TF had 
dimension order (time bins, frequencies) while TweetyNet has 
(frequencies, time bins)**

In [48]:
for tf_var_name, cnn_param in tf_var_cnn_param_map.items():
    array = tf.train.load_variable(str(ckpt_path), tf_var_name)
    # cnn_param.data = array

In [21]:
rnn_params = list(a_net.rnn.named_parameters())

In [22]:
for p in rnn_params:
    print(p[0], p[1].shape)

weight_ih_l0 torch.Size([2048, 512])
weight_hh_l0 torch.Size([2048, 512])
bias_ih_l0 torch.Size([2048])
bias_hh_l0 torch.Size([2048])
weight_ih_l0_reverse torch.Size([2048, 512])
weight_hh_l0_reverse torch.Size([2048, 512])
bias_ih_l0_reverse torch.Size([2048])
bias_hh_l0_reverse torch.Size([2048])
